# Building software with Hipfort

## Software environment

Any Fortran source file that incorporates other software through the `use` statement **must** be built with the same Fortran compiler as those other sources. It is then usually required to build Fortran software and interfaces with the **same compiler**. Building software with Hipfort requires the following software installed and ready to go:

* A GFortran compiler that is newer than 7.5.0. GFortran appears to be the only one that works at the moment, but this may change in future. 
* The ROCM stack installed. See [here](https://rocm.docs.amd.com/en/latest/) for latest installation info.
    * If you have an integrated graphics card from AMD you still might be able to use HIP. With my Radeon 680M i've had a good experience if I **don't** install the `amdgpu-dkms` package and just use the open source compute driver in the Linux kernel.
    * If hipfort is installed with the package manager during the installation of ROCM, you will probably need to remove it. Every unique Fortran compiler that you use must have a corresponding installation of Hipfort.
* The Hipfort library built with the same Fortran compiler mentioned above.
* Any other software that your application depends on.

### Hipfort installation on AMD platforms

Compiling and installing `hipfort` must be performed with the same compiler as you will build your application with. When compiling hipfort with gfortran the process should be fairly painless.

### Hipfort installation for NVIDIA platforms

While installing Hipfort for NVIDIA platforms **don't** set the environment variable `HIP_PLATFORM` to `nvidia`. You can set this environment flag later when building software. During compilation of your program the Hipfort Fortran compiler wrapper `hipfc` will examine `HIP_PLATFORM` and select either the `amdclang++` compiler for AMD devices or `nvcc` compiler for NVIDIA devices.

## Basic compilation with hipfc

Once `hipfort` is installed the compiler wrapper `hipfc` compiler wrapper can be used to compile Fortran sources. It supports the `-hipfort-compiler` option to select what Fortran compiler to use on the backend. Hipfc also supports C and C++ sources, and passes those sources to `hipcc` for compilation. 

### For AMD

The example below compiles mixed Fortran and HIP sources with an AMD graphics card. Notice that I'm passing both `C++` and `Fortran` sources to the compiler wrapper.

```bash
hipfc hip_utils.f90 math_utils.f90 tensoradd_hip_fptr.f90 kernel_code.cpp -o tensoradd.exe
```

Compilation in this way usually chooses the architecture for whatever AMD graphics card you are using. You can be specific though on which GPU architecture to compile for using the `--offload-arch` compiler flag. For an Mi250X GPU this would look like:

```bash
hipfc --offload-arch=gfx90a hip_utils.f90 math_utils.f90 tensoradd_hip_fptr.f90 kernel_code.cpp -o tensoradd.exe
```

### For NVIDIA

For compilation with an NVIDIA graphics card you need to set the environment variable `HIP_PLATFORM=nvidia`, you **also need** to choose the NVIDIA graphics card architecture via the `--offload-arch` compiler flag. For the same sources this works for my RTX 3060 GPU.

```bash
export HIP_PLATFORM=nvidia
hipfc --offload-arch=sm_86 hip_utils.f90 math_utils.f90 tensoradd_hip_fptr.f90 kernel_code.cpp -o tensoradd.exe 
```

### Linking with hipfc

When doing compile and link seperately it is usually a path of less pain to have the `hipfc` compiler wrapper pass the right flags on to the linker. Then it can link in the appropriate ROCM libraries.

## Building cross-platform software with CMake

HIP language support was formally introduced in CMake, version 3.21. This means you can set HIP as the language for kernel sources and the compiler will choose the necessary flags. Until version 3.28 of CMake though, the AMD backend was the only one that was supported by the HIP language with CMake. Since HIP on NVIDIA is just a thin wrapper over CUDA calls, an approach that seems to resolve the problem is to compile kernel sources with the **HIP language** on an AMD backend, and compile these **same kernel sources** as the **CUDA language** on an NVIDIA backend. Within a `CMakeLists.txt` file you can define the mutually exclusive pairs of macros (`__HIP_PLATFORM_HCC__`, `__HIP_PLATFORM_AMD__`) and (`__HIP_PLATFORM_NVCC__`,`__HIP_PLATFORM_NVIDIA__`) as switches for choosing the backend libraries to use from `hip_runtime.h`.

### HIP language

You can enable the `HIP` language in the `project` description and that will enable HIP-specific CMake variables that you can work with. In the main <a href="../CMakeLists.txt">../CMakeLists.txt</a> file we add the `HIP`, `CXX`, and `Fortran` languages for the build system to support.

```CMAKE
project(hipfort_course VERSION 1.0.0
    DESCRIPTION "Example codes to accompany a course in hipfort"
    LANGUAGES HIP CXX Fortran
)
```

### Finding HIP

The `find_package` directive in CMAKE will scan for a HIP installation. In <a href="../CMakeLists.txt">../CMakeLists.txt</a> we use it to require that HIP is found.

```CMAKE
# HIP
find_package(hip REQUIRED)
```

If HIP is found then the variable `hip_FOUND` will be set to TRUE.


### Associate the HIP language with kernel sources

CMake uses the file extension to choose which language to use when setting up sources for compilation. Files with the `.hip` extension will be treated as `HIP` sources, and compiled with ROCM's internal `clang++` compiler. You can modify which language gets chosen for a particular source file by using `set_source_files_properties` to set the language for a source file or collection of source files. In the example below we set the HIP language for a list of sources called `KERNEL_FILES`.

```CMAKE
# Set the language for any kernel files
set(KERNEL_FILES
    ${CMAKE_CURRENT_SOURCE_DIR}/kernel_code.cpp
)
set_source_files_properties(${KERNEL_FILES} PROPERTIES LANGUAGE HIP)
```

### CUDA language

### Cross platform installation